In [ ]:
import scanpy as sc
import decoupler as dc
# Add src to path
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join("..", "src")))
from utils import load_sampled_data

In [ ]:
test = sc.datasets.pbmc3k_processed()

In [10]:
type(test.var)

pandas.core.frame.DataFrame

In [11]:
X = load_sampled_data("../data/processed/preprocessed_landmark.csv", sample_size=1000)

In [15]:
X.columns.to_list()

['AARS',
 'ABCF1',
 'ABL1',
 'ACAA1',
 'ACAT2',
 'ACLY',
 'ADAM10',
 'ADH5',
 'PARP1',
 'ADRB2',
 'AGL',
 'AKT1',
 'ALAS1',
 'ALDOA',
 'ALDOC',
 'SLC25A4',
 'ANXA7',
 'APBB2',
 'BIRC2',
 'BIRC5',
 'APOE',
 'APP',
 'FAS',
 'RHOA',
 'ARHGAP1',
 'ASAH1',
 'ATF1',
 'ATP1B1',
 'ALDH7A1',
 'ATP6V0B',
 'BAD',
 'BAX',
 'CCND1',
 'BCL2',
 'BDH1',
 'BID',
 'BLMH',
 'BLVRA',
 'BMP4',
 'BNIP3',
 'BNIP3L',
 'BPHL',
 'BRCA1',
 'BTK',
 'BUB1B',
 'C5',
 'DDR1',
 'CALM3',
 'CALU',
 'CAPN1',
 'CAST',
 'CASP2',
 'CASP3',
 'CASP7',
 'CASP10',
 'CAT',
 'CBLB',
 'CBR1',
 'CBR3',
 'CCNA2',
 'CCNB1',
 'CCND3',
 'CCNF',
 'CCNH',
 'SCARB1',
 'CD40',
 'CD44',
 'CD58',
 'ADGRE5',
 'CDK1',
 'CDC20',
 'CDC25A',
 'CDC25B',
 'CDC42',
 'CDH3',
 'CDK2',
 'CDK4',
 'CDK6',
 'CDK7',
 'CDKN1A',
 'CDKN1B',
 'CDKN2A',
 'CEBPA',
 'CEBPD',
 'CENPE',
 'CETN3',
 'CHEK1',
 'CHN1',
 'CIRBP',
 'CLTB',
 'CLTC',
 'COL1A1',
 'COL4A1',
 'CREB1',
 'CRK',
 'CRKL',
 'CRYZ',
 'CSK',
 'CSNK1A1',
 'CSNK1E',
 'CSNK2A2',
 'CSRP1',
 'CTNND1',
 

In [12]:
net = dc.get_collectri(organism="human", split_complexes=False)

In [ ]:
import pandas as pd
import logging

# Define expected metadata columns
metadata_cols = {"cell_mfc_name", "viability", "pert_dose"}
missing_cols = metadata_cols - set(X.columns)
if missing_cols:
    raise ValueError(f"Missing expected metadata columns: {missing_cols}")

# Separate metadata from gene expression data
metadata = X[list(metadata_cols)]
gene_expression = X.drop(columns=metadata_cols)
print("Gene expression data shape:", gene_expression.shape)

# Determine shared genes between the network and gene expression data
shared_genes = set(net["target"]).intersection(gene_expression.columns)
if not shared_genes:
    raise ValueError(
        "No shared genes found between network and gene expression matrix!"
    )
logging.debug(f"Number of shared genes: {len(shared_genes)}")

# Filter the network and gene expression data to include only shared genes
net_filtered = net[net["target"].isin(shared_genes)]
logging.debug(f"Filtered network has {len(net_filtered)} interactions.")
gene_expression = gene_expression[list(shared_genes)]

# Create an AnnData object for the gene expression data
adata = sc.AnnData(
    X=gene_expression.values,
    obs=pd.DataFrame(index=gene_expression.index),
    var=pd.DataFrame(index=gene_expression.columns),
)

In [ ]:
type(adata.var)

In [ ]:
dc.run_mlm(
    mat=adata, net=net, source="source", target="target", weight="weight", verbose=True, use_raw=False
)

In [ ]:
adata.obsm["mlm_estimate"]